In [1]:
import numpy as np
import pandas as pd
import os
import glob


In [2]:
# load and name each file by its file name in folder (without extension)
path = 'files/'
var_list = []
countries = []

for f in glob.glob(os.path.join(path, '*.csv')):
    variable_name = os.path.splitext(os.path.basename(f))[0]
    var_list.append(variable_name)
    globals()[variable_name] = pd.read_csv(f, parse_dates=['TIME'],index_col='TIME') # load all files separately, parse date column 'TIME' as index of datetime
    globals()[variable_name] = globals()[variable_name][~(globals()[variable_name].index < '1980-01-01')]
    # create a list containing all countries from all files
    for country in globals()[variable_name].LOCATION:
        countries.append(country)
        countries = list(dict.fromkeys(countries))
    



In [3]:
# change time formatting
for var in var_list:
    if var[-1] == 'M':
        globals()[var].index = globals()[var].index.strftime("%Y-%m")
    else:
        globals()[var].index = globals()[var].index.to_period('D').strftime('%YQ%q')

        

In [4]:
# save formal variable description to use as main sheet name
variable_description = {}

for var in var_list:    
    variable_description[var] = str(globals()[var].iloc[0][1] + ' ' + globals()[var].iloc[0][2] + ' ' + globals()[var].iloc[0][3])

In [5]:
# transform with pivot and complete missing countries
for var in var_list:
    # pivot wider, columns=LOCATION, values=Value
    globals()[var] = globals()[var].pivot(columns='LOCATION', values='Value')
    # add missing countries to all variables
    for country in countries:
        if country in globals()[var].columns:
            continue
        else:
            globals()[var][country] = np.nan
    globals()[var] = globals()[var].reindex(sorted(globals()[var].columns), axis=1)



In [6]:
# create a dictionary with info about each variable - start date and end date
first_last_date = {}

for var in var_list:
    first_last_date[var] = {'index':['first', 'last', 'in between']}
    for country in countries:
        if globals()[var][country].sum() != 0:
            try:
                ind = globals()[var][country][globals()[var][country].first_valid_index():globals()[var][country].last_valid_index()].isna()
                ind.drop(ind[ind==False].index, inplace=True)
                ind = ind.index
                ind = list(ind)
                ind = ','.join(ind)
                first_last_date[var][country] = [globals()[var][country].first_valid_index(), globals()[var][country].last_valid_index(), ind]
            except:
                continue
        else:
            first_last_date[var][country] = ['missing', 'missing', 'missing']  

In [7]:
for var in var_list:
    path = "%s.xlsx" % var
    main_sheet = variable_description[var]
    with pd.ExcelWriter(path) as writer:
        globals()[var].to_excel(writer, sheet_name=variable_description[var], index=True)
        first_last = pd.DataFrame.from_dict(first_last_date[var], orient='columns')
        first_last = first_last.reindex(sorted(first_last.columns), axis=1)
        first_last = first_last.T
        first_last.columns = first_last.iloc[np.where(first_last[0] == 'first')[0][0]]
        first_last = first_last.drop(first_last.index[-1])        
        first_last.to_excel(writer, sheet_name="first_last", index=True)
    